In [10]:
import cv2  
import os
from PIL import Image
from tensorflow import keras
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import normalize
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import accuracy_score

In [2]:
INPUT_SIZE = 64
image_directory = 'datasets/'

no_tumor_images=os.listdir(image_directory+ 'crop_no/')
yes_tumor_images=os.listdir(image_directory+ 'crop_yes/')
dataset=[]
label=[]

In [3]:
for i , image_name in enumerate(no_tumor_images) :
    if(image_name.split('.')[1] == 'jpg') :
        image=cv2.imread(image_directory+ 'crop_no/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(0)

for i , image_name in enumerate(yes_tumor_images) :
    if(image_name.split('.')[1] == 'jpg') :
        image=cv2.imread(image_directory+ 'crop_yes/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE,INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(1)



In [4]:
np.array(dataset)
label = np.array(label)
x_train, x_test, y_train, y_test = train_test_split(dataset, label, test_size = 0.2 )

x_train = np.array(x_train, dtype=float)
x_test= np.array(x_test, dtype=float)
print(x_train.shape)

x_train_normalized=x_train/255 # chia khoảng từ 0-1
x_test_normalized=x_test/255 # chia khoảng từ 0-1

y_train=to_categorical(y_train, num_classes=2) # đánh nhãn label theo dạng one-hot vector
y_test=to_categorical(y_test, num_classes=2) 


print(x_train)
print(x_train_normalized)


(2000, 64, 64, 3)
[[[[  1.   1.   1.]
   [  1.   1.   1.]
   [  1.   1.   1.]
   ...
   [  1.   1.   1.]
   [  1.   1.   1.]
   [  1.   1.   1.]]

  [[  1.   1.   1.]
   [  1.   1.   1.]
   [  1.   1.   1.]
   ...
   [  1.   1.   1.]
   [  1.   1.   1.]
   [  1.   1.   1.]]

  [[  1.   1.   1.]
   [  1.   1.   1.]
   [  1.   1.   1.]
   ...
   [  1.   1.   1.]
   [  1.   1.   1.]
   [  1.   1.   1.]]

  ...

  [[  4.   4.   4.]
   [  0.   0.   0.]
   [  2.   2.   2.]
   ...
   [  1.   1.   1.]
   [  1.   1.   1.]
   [  1.   1.   1.]]

  [[  0.   0.   0.]
   [  0.   0.   0.]
   [  4.   4.   4.]
   ...
   [  1.   1.   1.]
   [  1.   1.   1.]
   [  2.   2.   2.]]

  [[  1.   1.   1.]
   [  3.   3.   3.]
   [  0.   0.   0.]
   ...
   [  1.   1.   1.]
   [  1.   1.   1.]
   [  1.   1.   1.]]]


 [[[ 85.  85.  85.]
   [ 71.  71.  71.]
   [ 72.  72.  72.]
   ...
   [  0.   0.   0.]
   [  0.   0.   0.]
   [  0.   0.   0.]]

  [[ 60.  60.  60.]
   [ 63.  63.  63.]
   [ 94.  94.  94.]
   ...
   

In [9]:
model=Sequential() # tuần tự nghĩa là các lớp được thêm vào theo thứ tự câu lệnh
model.add(Conv2D(32, (3, 3),input_shape=(INPUT_SIZE, INPUT_SIZE, 3))) # gồm 3 lớp convolution xen kẽ với lớp pooling
model.add(Activation('relu')) #công thức tính outpout 1 layer là A^l = activation(W^l x A^l-1 + B^l), relu :  max(0,x) 
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(32, (3, 3), kernel_initializer='he_uniform'))
model.add(Activation('relu')) # ví sao dùng activation: tạo sự phi tuyến tính + giữ output trong khoảng nhất định (0,1)
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3),kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten()) # duỗi thẳng 
model.add(Dense(64)) # đưa qua 1 lớp 64 neuron
model.add(Activation('relu'))
model.add(Dropout(0.2)) 
model.add(Dense(2)) # đưa qua lớp 2 neuron là : yes no
model.add(Activation('sigmoid')) # 1/(1 + e^-x)

c:\Users\Vu The Van\miniconda3\envs\test\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
optimizer1 = tf.keras.optimizers.Adam(learning_rate=1e-3) 
model.compile(
    loss='binary_crossentropy',
    optimizer=optimizer1,
    metrics=['accuracy'] 
)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
import log
file_logging_callback  = log.FileLoggingCallback()

model.fit(
    x_train_normalized, 
    y_train, 
    batch_size=16,
    verbose=1,
    epochs=10, 
    validation_data=(x_test_normalized, y_test),
    shuffle=True,
    callbacks=[file_logging_callback, reduce_lr, early_stopping]
)


#Lưu mô hình
model.save('Braintumor10EpochsCategorical.h5')

Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.5981 - loss: 0.6933 - val_accuracy: 0.6760 - val_loss: 0.6085 - learning_rate: 0.0010
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.6811 - loss: 0.5830 - val_accuracy: 0.6580 - val_loss: 0.5692 - learning_rate: 0.0010
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.7451 - loss: 0.5119 - val_accuracy: 0.8040 - val_loss: 0.4466 - learning_rate: 0.0010
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8139 - loss: 0.4041 - val_accuracy: 0.7960 - val_loss: 0.4182 - learning_rate: 0.0010
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8487 - loss: 0.3399 - val_accuracy: 0.7460 - val_loss: 0.4571 - learning_rate: 0.0010
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8641 - loss: 0.3085 - val_accuracy: 0.8080 - val_loss: 0.3661 - learning_rate: 0.0010
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9096 - loss: 0.

In [12]:
from tensorflow.keras.models import load_model
from sklearn.metrics import f1_score
import numpy as np

# Load model
model = load_model('Braintumor10EpochsCategorical.h5')

# Dự đoán trên dữ liệu kiểm tra
y_pred = model.predict(x_test_normalized)

# Chuyển đổi xác suất thành nhãn phân loại
y_pred_classes = np.argmax(y_pred, axis=1)

# Nếu y_test là one-hot encoded, chuyển đổi nó thành nhãn phân loại
y_test_classes = np.argmax(y_test, axis=1)



# Tính toán F1 Score
score = f1_score(y_test_classes, y_pred_classes)
print(f"F1 Score: {score}")

# Tính accuracy
accuracy = accuracy_score(y_test_classes, y_pred_classes)
print(f"Accuracy của mô hình : {accuracy}")


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
F1 Score: 0.9370629370629371
Accuracy của mô hình : 0.946
